<a href="https://colab.research.google.com/github/simonskyjumpp/bigdataanalysis/blob/main/bike_store_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving bike_store_database.xlsx to bike_store_database.xlsx


In [ ]:
import pandas as pd

file_name = "bike_store_database.xlsx"

In [ ]:
dfs = pd.read_excel(file_name, sheet_name=None)

In [ ]:
print("Sheets in the file:", dfs.keys())

Sheets in the file: dict_keys(['brands', 'orders', 'products', 'staffs', 'stocks', 'stores', 'customers', 'order_items', 'categories'])


In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect("file_name")

In [ ]:
for sheet, df in dfs.items():
    df.to_sql(sheet, conn, index=False, if_exists="replace")

In [ ]:
print("Database created with tables:", dfs.keys())

Database created with tables: dict_keys(['brands', 'orders', 'products', 'staffs', 'stocks', 'stores', 'customers', 'order_items', 'categories'])


In [ ]:
query = "SELECT * FROM orders LIMIT 5;"

df_sql = pd.read_sql(query, conn)
print(df_sql)

   order_id  customer_id  order_status           order_date  \
0         1          259             4  2016-01-01 00:00:00   
1         2         1212             4  2016-01-01 00:00:00   
2         3          523             4  2016-01-02 00:00:00   
3         4          175             4  2016-01-03 00:00:00   
4         5         1324             4  2016-01-03 00:00:00   

         required_date shipped_date  store_id  staff_id  
0  2016-01-03 00:00:00   2016-01-03         1         2  
1  2016-01-04 00:00:00   2016-01-03         2         6  
2  2016-01-05 00:00:00   2016-01-03         2         7  
3  2016-01-04 00:00:00   2016-01-05         1         3  
4  2016-01-06 00:00:00   2016-01-06         2         6  


In [ ]:
query1 = """
SELECT
    o.order_id,
    c.first_name,
    c.last_name,
    o.order_status,
    o.order_date
FROM
    orders o
JOIN
    Customers c ON o.customer_id = c.customer_id
LIMIT 5;
"""

df_customer_orders = pd.read_sql(query1, conn)
print("Query 1 Results:")
print(df_customer_orders)

Query 1 Results:
   order_id first_name  last_name  order_status           order_date
0         1  Johnathan  Velazquez             4  2016-01-01 00:00:00
1         2  Jaqueline   Cummings             4  2016-01-01 00:00:00
2         3     Joshua  Robertson             4  2016-01-02 00:00:00
3         4       Nova       Hess             4  2016-01-03 00:00:00
4         5       Arla      Ellis             4  2016-01-03 00:00:00


In [ ]:
query2 = """
SELECT
    o.order_id,
    c.email,
    o.order_status,
    o.order_date
FROM
    orders o
JOIN
    Customers c ON o.customer_id = c.customer_id
LIMIT 5;
"""

df_customer_orders = pd.read_sql(query2, conn)
print("Query 2 Results:")
print(df_customer_orders)

Query 2 Results:
   order_id                            email  order_status  \
0         1  johnathan.velazquez@hotmail.com             4   
1         2   jaqueline.cummings@hotmail.com             4   
2         3       joshua.robertson@gmail.com             4   
3         4                nova.hess@msn.com             4   
4         5             arla.ellis@yahoo.com             4   

            order_date  
0  2016-01-01 00:00:00  
1  2016-01-01 00:00:00  
2  2016-01-02 00:00:00  
3  2016-01-03 00:00:00  
4  2016-01-03 00:00:00  


In [ ]:
query3 = """
SELECT
    o.customer_id,
    c.first_name,
    c.last_name,
    c.email,
    AVG(p.list_price) as average_listing_price
FROM
    orders o
JOIN
    Customers c ON o.customer_id = c.customer_id
JOIN
    order_items oi ON o.order_id = oi.order_id
JOIN
    products p ON oi.product_id = p.product_id
GROUP BY
    o.customer_id, c.first_name, c.last_name
HAVING
    SUM(oi.quantity) > 3;
"""

df_high_value_customers = pd.read_sql(query3, conn)
print("Query 3 Results:")
print(df_high_value_customers)

Query 3 Results:
     customer_id  first_name last_name                     email  \
0              1       Debra     Burks     debra.burks@yahoo.com   
1              2       Kasha      Todd      kasha.todd@yahoo.com   
2              3      Tameka    Fisher     tameka.fisher@aol.com   
3              4       Daryl    Spence      daryl.spence@aol.com   
4              5  Charolette      Rice   charolette.rice@msn.com   
..           ...         ...       ...                       ...   
949         1439     Florrie    Little  florrie.little@yahoo.com   
950         1440      Ernest   Rollins  ernest.rollins@yahoo.com   
951         1441      Jamaal  Morrison   jamaal.morrison@msn.com   
952         1442      Cassie     Cline    cassie.cline@gmail.com   
953         1443      Lezlie      Lamb     lezlie.lamb@gmail.com   

     average_listing_price  
0              2096.173636  
1              1403.690000  
2              1371.528462  
3              1703.213333  
4              1680.7

(7, 2)


In [ ]:
# a. Total sales amount per product category
query_a = """
SELECT
    p.category_id,
    SUM(oi.quantity * p.list_price) AS total_sales_amount
FROM
    Products p
JOIN
    Order_Items oi ON p.product_id = oi.product_id
GROUP BY
    p.category_id;
"""
df_sales_amount_per_product_category = pd.read_sql(query_a, conn)
print("Query a Results:")
print(df_sales_amount_per_product_category)

Query a Results:
   category_id  total_sales_amount
0            1           327888.21
1            2           438506.87
2            3          1109151.04
3            4           799874.60
4            5          1020236.85
5            6          3030775.71
6            7          1852555.60


In [ ]:
# b. Average list price
query_b = """
SELECT
    AVG(list_price) AS average_list_price
FROM
    Products;
"""
df_average_list_price = pd.read_sql(query_b, conn)
print("Query b Results:")
print(df_average_list_price)

Query b Results:
   average_list_price
0         1520.591402


In [ ]:
# c. Top 10 selling products (by quantity)
query_c = """
SELECT
    p.product_name,
    SUM(oi.quantity) AS total_quantity_sold
FROM
    Products p
JOIN
    Order_Items oi ON p.product_id = oi.product_id
GROUP BY
    p.product_name
ORDER BY
    total_quantity_sold DESC
LIMIT 10;
"""
df_top_ten_selling_products = pd.read_sql(query_c, conn)
print("Query c Results:")
print(df_top_ten_selling_products)

Query c Results:
                                    product_name  total_quantity_sold
0             Electra Cruiser 1 (24-Inch) - 2016                  296
1           Electra Townie Original 7D EQ - 2016                  290
2             Electra Townie Original 21D - 2016                  289
3  Electra Girl's Hawaii 1 (16-inch) - 2015/2016                  269
4          Surly Ice Cream Truck Frameset - 2016                  167
5                       Trek Slash 8 27.5 - 2016                  154
6  Electra Girl's Hawaii 1 (20-inch) - 2015/2016                  154
7                    Surly Straggler 650b - 2016                  151
8         Electra Townie Original 7D - 2015/2016                  148
9                         Surly Straggler - 2016                  147


In [ ]:
# d. Orders sold by staff
query_d = """
SELECT
    s.first_name,
    s.last_name,
    s.staff_id,
    COUNT(o.order_id) AS total_orders
FROM
    Staffs s
JOIN
    Orders o ON s.staff_id = o.staff_id
GROUP BY
    s.staff_id, s.first_name, s.last_name
ORDER BY
    total_orders DESC;
"""
df_orders_sold_by_staff = pd.read_sql(query_d, conn)
print("Query d Results:")
print(df_orders_sold_by_staff)

Query d Results:
  first_name last_name  staff_id  total_orders
0  Marcelene     Boyer         6           553
1     Venita    Daniel         7           540
2      Genna   Serrano         3           184
3     Mireya  Copeland         2           164
4       Kali    Vargas         8            88
5      Layla   Terrell         9            86


In [ ]:
# e. Average shipping time by stores
query_e = """
SELECT
    s.store_name,
    AVG(JULIANDAY(o.shipped_date) - JULIANDAY(o.order_date)) AS average_shipping_time
FROM
    Orders o
JOIN
    Stores s ON o.store_id = s.store_id
GROUP BY
    s.store_id, s.store_name;
"""
df_average_shipping_time_by_stores = pd.read_sql(query_e, conn)
print("Query e Results:")
print(df_average_shipping_time_by_stores)

Query e Results:
         store_name  average_shipping_time
0  Santa Cruz Bikes               2.045775
1     Baldwin Bikes               1.972522
2     Rowlett Bikes               1.922535


In [ ]:
conn.close()